## MLP Starter Kit for Kaggle Digit Recognizer

#### Imports and inits

In [1]:
# imports
import h2o 
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.grid.grid_search import H2OGridSearch

In [2]:
# display matplotlib graphics in notebook
%matplotlib inline

In [3]:
# start and connect to h2o server
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,11 days 11 hours 6 mins
H2O cluster version:,3.10.4.4
H2O cluster version age:,2 months and 7 days
H2O cluster name:,H2O_from_python_yataolu_sxebkd
H2O cluster total nodes:,1
H2O cluster free memory:,1.434 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"
H2O connection url:,http://localhost:54321
H2O connection proxy:,None


## Import data

In [11]:
train = h2o.import_file(path= 'train.csv', col_types=col_types) # multi-threaded import
test = h2o.import_file(path= 'test1.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [12]:
# define input variable measurement levels 
# strings automatically parsed as enums (nominal)
# numbers automatically parsed as numeric
col_types = {'label': 'enum'}

In [13]:
train.describe()

Rows:42000
Cols:785




,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,pixel40,pixel41,pixel42,pixel43,pixel44,pixel45,pixel46,pixel47,pixel48,pixel49,pixel50,pixel51,pixel52,pixel53,pixel54,pixel55,pixel56,pixel57,pixel58,pixel59,pixel60,pixel61,pixel62,pixel63,pixel64,pixel65,pixel66,pixel67,pixel68,pixel69,pixel70,pixel71,pixel72,pixel73,pixel74,pixel75,pixel76,pixel77,pixel78,pixel79,pixel80,pixel81,pixel82,pixel83,pixel84,pixel85,pixel86,pixel87,pixel88,pixel89,pixel90,pixel91,pixel92,pixel93,pixel94,pixel95,pixel96,pixel97,pixel98,pixel99,pixel100,pixel101,pixel102,pixel103,pixel104,pixel105,pixel106,pixel107,pixel108,pixel109,pixel110,pixel111,pixel112,pixel113,pixel114,pixel115,pixel116,pixel117,pixel118,pixel119,pixel120,pixel121,pixel122,pixel123,pixel124,pixel125,pixel126,pixel127,pixel128,pixel129,pixel130,pixel131,pixel132,pixel133,pixel134,pixel135,pixel136,pixel137,pixel138,pixel139,pixel140,pixel141,pixel142,pixel143,pixel144,pixel145,pixel146,pixel147,pixel148,pixel149,pixel150,pixel151,pixel152,pixel153,pixel154,pixel155,pixel156,pixel157,pixel158,pixel159,pixel160,pixel161,pixel162,pixel163,pixel164,pixel165,pixel166,pixel167,pixel168,pixel169,pixel170,pixel171,pixel172,pixel173,pixel174,pixel175,pixel176,pixel177,pixel178,pixel179,pixel180,pixel181,pixel182,pixel183,pixel184,pixel185,pixel186,pixel187,pixel188,pixel189,pixel190,pixel191,pixel192,pixel193,pixel194,pixel195,pixel196,pixel197,pixel198,pixel199,pixel200,pixel201,pixel202,pixel203,pixel204,pixel205,pixel206,pixel207,pixel208,pixel209,pixel210,pixel211,pixel212,pixel213,pixel214,pixel215,pixel216,pixel217,pixel218,pixel219,pixel220,pixel221,pixel222,pixel223,pixel224,pixel225,pixel226,pixel227,pixel228,pixel229,pixel230,pixel231,pixel232,pixel233,pixel234,pixel235,pixel236,pixel237,pixel238,pixel239,pixel240,pixel241,pixel242,pixel243,pixel244,pixel245,pixel246,pixel247,pixel248,pixel249,pixel250,pixel251,pixel252,pixel253,pixel254,pixel255,pixel256,pixel257,pixel258,pixel259,pixel260,pixel261,pixel262,pixel263,pixel264,pixel265,pixel266,pixel267,pixel268,pixel269,pixel270,pixel271,pixel272,pixel273,pixel274,pixel275,pixel276,pixel277,pixel278,pixel279,pixel280,pixel281,pixel282,pixel283,pixel284,pixel285,pixel286,pixel287,pixel288,pixel289,pixel290,pixel291,pixel292,pixel293,pixel294,pixel295,pixel296,pixel297,pixel298,pixel299,pixel300,pixel301,pixel302,pixel303,pixel304,pixel305,pixel306,pixel307,pixel308,pixel309,pixel310,pixel311,pixel312,pixel313,pixel314,pixel315,pixel316,pixel317,pixel318,pixel319,pixel320,pixel321,pixel322,pixel323,pixel324,pixel325,pixel326,pixel327,pixel328,pixel329,pixel330,pixel331,pixel332,pixel333,pixel334,pixel335,pixel336,pixel337,pixel338,pixel339,pixel340,pixel341,pixel342,pixel343,pixel344,pixel345,pixel346,pixel347,pixel348,pixel349,pixel350,pixel351,pixel352,pixel353,pixel354,pixel355,pixel356,pixel357,pixel358,pixel359,pixel360,pixel361,pixel362,pixel363,pixel364,pixel365,pixel366,pixel367,pixel368,pixel369,pixel370,pixel371,pixel372,pixel373,pixel374,pixel375,pixel376,pixel377,pixel378,pixel379,pixel380,pixel381,pixel382,pixel383,pixel384,pixel385,pixel386,pixel387,pixel388,pixel389,pixel390,pixel391,pixel392,pixel393,pixel394,pixel395,pixel396,pixel397,pixel398,pixel399,pixel400,pixel401,pixel402,pixel403,pixel404,pixel405,pixel406,pixel407,pixel408,pixel409,pixel410,pixel411,pixel412,pixel413,pixel414,pixel415,pixel416,pixel417,pixel418,pixel419,pixel420,pixel421,pixel422,pixel423,pixel424,pixel425,pixel426,pixel427,pixel428,pixel429,pixel430,pixel431,pixel432,pixel433,pixel434,pixel435,pixel436,pixel437,pixel438,pixel439,pixel440,pixel441,pixel442,pixel443,pixel444,pixel445,pixel446,pixel447,pixel448,pixel449,pixel450,pixel451,pixel452,pixel453,pixel454,pixel455

In [14]:
# split into 40% training, 30% validation, and 30% test
train, valid = train.split_frame([0.8])

In [15]:
# assign target and inputs
y = 'label'
X = [name for name in train.columns if name != y]
print(y)
print(X)

label
[u'pixel0', u'pixel1', u'pixel2', u'pixel3', u'pixel4', u'pixel5', u'pixel6', u'pixel7', u'pixel8', u'pixel9', u'pixel10', u'pixel11', u'pixel12', u'pixel13', u'pixel14', u'pixel15', u'pixel16', u'pixel17', u'pixel18', u'pixel19', u'pixel20', u'pixel21', u'pixel22', u'pixel23', u'pixel24', u'pixel25', u'pixel26', u'pixel27', u'pixel28', u'pixel29', u'pixel30', u'pixel31', u'pixel32', u'pixel33', u'pixel34', u'pixel35', u'pixel36', u'pixel37', u'pixel38', u'pixel39', u'pixel40', u'pixel41', u'pixel42', u'pixel43', u'pixel44', u'pixel45', u'pixel46', u'pixel47', u'pixel48', u'pixel49', u'pixel50', u'pixel51', u'pixel52', u'pixel53', u'pixel54', u'pixel55', u'pixel56', u'pixel57', u'pixel58', u'pixel59', u'pixel60', u'pixel61', u'pixel62', u'pixel63', u'pixel64', u'pixel65', u'pixel66', u'pixel67', u'pixel68', u'pixel69', u'pixel70', u'pixel71', u'pixel72', u'pixel73', u'pixel74', u'pixel75', u'pixel76', u'pixel77', u'pixel78', u'pixel79', u'pixel80', u'pixel81', u'pixel82', u'pixel

In [19]:
# set target to factor - for multinomial classification
train[y] = train[y].asfactor()
valid[y] = valid[y].asfactor()

## Train MLP using random grid search

In [18]:
# NN with random hyperparameter search
# train many different NN models with random hyperparameters
# and select best model based on validation error

# define random grid search parameters
hyper_parameters = {'hidden': [[500, 500], [250, 250, 250, 250], [1000, 500], [500, 1000], [1000, 500, 250], [1000, 1000]],
                    'l1':[s/1e4 for s in range(0, 1000, 100)],
                    'l2':[s/1e5 for s in range(0, 1000, 100)],
                    'input_dropout_ratio':[s/1e2 for s in range(0, 20, 2)]}

# define search strategy
search_criteria = {'strategy':'RandomDiscrete',
                   'max_models':50,
                   'max_runtime_secs':3600}

# initialize grid search
gsearch = H2OGridSearch(H2ODeepLearningEstimator,
                        hyper_params=hyper_parameters,
                        search_criteria=search_criteria)

# execute training w/ grid search
gsearch.train(x=X,
              y=y,
              training_frame=train,
              validation_frame=valid, 
              activation='RectifierWithDropout', 
              epochs=8000, 
              stopping_rounds=20,
              
              sparse=True, # handles data w/ many zeros more efficiently                     
              ignore_const_cols=True,              
              adaptive_rate=True)

# view detailed results at http://host:ip/flow/index.html

deeplearning Grid Build progress: |███████████████████████████████████████| 100%


#### Select best model

In [20]:
# show grid search results
gsearch.show()

# select best model
mnist_model = gsearch.get_grid()[0]

# print model information
mnist_model

# hit-ratio = ((TP + TN)/(TP + TN + FP + FN)), for two-classes


          hidden input_dropout_ratio    l1     l2  \
0    [1000, 500]                0.08  0.08    0.0   
1    [500, 1000]                0.12  0.07  0.004   

                                                              model_ids  \
0  Grid_DeepLearning_py_5_sid_a1f1_model_python_1497195175535_6_model_0   
1  Grid_DeepLearning_py_5_sid_a1f1_model_python_1497195175535_6_model_1   

              logloss  
0  2.2822471783940714  
1  2.3034782904035533  
Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  Grid_DeepLearning_py_5_sid_a1f1_model_python_1497195175535_6_model_0


ModelMetricsMultinomial: deeplearning
** Reported on train data. **

MSE: 0.805028177951
RMSE: 0.897233625067
LogLoss: 2.28199905218
Mean Per-Class Error: 0.878265252684
Confusion Matrix: vertical: actual; across: predicted



0,1,2,3,4,5,6,7,8,9,Error,Rate
0.0,0.0,0.0,0.0,520.0,0.0,0.0,456.0,0.0,0.0,1.0,976 / 976
0.0,0.0,0.0,0.0,55.0,0.0,0.0,1030.0,0.0,0.0,1.0,"1,085 / 1,085"
0.0,0.0,0.0,0.0,813.0,0.0,0.0,166.0,0.0,0.0,1.0,979 / 979
0.0,0.0,0.0,0.0,490.0,0.0,0.0,542.0,0.0,0.0,1.0,"1,032 / 1,032"
0.0,0.0,0.0,0.0,311.0,0.0,0.0,694.0,0.0,0.0,0.6905473,"694 / 1,005"
0.0,0.0,0.0,0.0,598.0,0.0,0.0,324.0,0.0,0.0,1.0,922 / 922
0.0,0.0,0.0,0.0,755.0,0.0,0.0,215.0,0.0,0.0,1.0,970 / 970
0.0,0.0,0.0,0.0,91.0,0.0,0.0,897.0,0.0,0.0,0.0921053,91 / 988
0.0,0.0,0.0,0.0,288.0,0.0,0.0,665.0,0.0,0.0,1.0,953 / 953
0.0,0.0,0.0,0.0,162.0,0.0,0.0,819.0,0.0,0.0,1.0,981 / 981


Top-10 Hit Ratios: 


k,hit_ratio
1,0.1221312
2,0.1988677
3,0.3588110
4,0.5064200
5,0.6153068
6,0.6837529
7,0.7407744
8,0.8634112
9,0.9033465
10,1.0



ModelMetricsMultinomial: deeplearning
** Reported on validation data. **

MSE: 0.804977057869
RMSE: 0.897205137006
LogLoss: 2.28224717839
Mean Per-Class Error: 0.880212787789
Confusion Matrix: vertical: actual; across: predicted



...

Top-10 Hit Ratios: 


k,hit_ratio
1,0.1237089
2,0.1996157
3,0.3611578
4,0.5134519
5,0.6178237
6,0.6895268
7,0.7476580
8,0.8621187
9,0.9000720
10,1.0


Scoring History: 


,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_classification_error,validation_rmse,validation_logloss,validation_classification_error
,2017-06-22 22:43:05,0.000 sec,None,0.0,0,0.0,nan,nan,nan,nan,nan,nan
,2017-06-22 22:43:12,16.999 sec,512 obs/sec,0.0943161,1,3176.0,0.8972336,2.2819991,0.8778688,0.8972051,2.2822472,0.8762911
,2017-06-22 22:44:56,1 min 59.802 sec,1680 obs/sec,5.0163628,52,168921.0,0.8999030,2.3021878,0.8956627,0.8998585,2.3017368,0.9031948
,2017-06-22 22:46:21,3 min 24.929 sec,1925 obs/sec,10.1220526,105,340850.0,0.8999927,2.3032108,0.9019310,0.9000352,2.3036250,0.9045160
,2017-06-22 22:47:49,4 min 52.352 sec,2083 obs/sec,15.8072103,164,532292.0,0.8998668,2.3032724,0.8903043,0.8997145,2.3018268,0.8832573
---,---,---,---,---,---,---,---,---,---,---,---,---
,2017-06-22 23:35:47,52 min 50.556 sec,3052 obs/sec,258.1088080,2683,8691556.0,0.9040062,2.5340164,0.8956627,0.9057126,2.5428123,0.9031948
,2017-06-22 23:37:13,54 min 16.396 sec,3058 obs/sec,265.7096870,2762,8947508.0,0.9005530,2.3759879,0.9019310,0.8997930,2.3675046,0.9045160
,2017-06-22 23:38:35,55 min 38.097 sec,3066 obs/sec,273.1172715,2839,9196951.0,0.9005176,2.3728875,0.8983925,0.9006369,2.3738445,0.9005525
,2017-06-22 23:39:56,56 min 59.787 sec,3071 obs/sec,280.2580923,2913,9437411.0,0.8999102,2.3082283,0.9019310,0.8999000,2.3081099,0.9045160



See the whole table with table.as_data_frame()


#### Best known H2O MLP for MNIST
* Can you train it?

In [21]:
"""

best_model = H2ODeepLearningEstimator(
                activation = 'RectifierWithDropout', 
                hidden = [1024,1024,2048],
                epochs = 8000, 
                l1 = 1e-5, 
                input_dropout_ratio = 0.2,
                train_samples_per_iteration = -1, 
                classification_stop = -1)
                
"""

"\n\nbest_model = H2ODeepLearningEstimator(\n                activation = 'RectifierWithDropout', \n                hidden = [1024,1024,2048],\n                epochs = 8000, \n                l1 = 1e-5, \n                input_dropout_ratio = 0.2,\n                train_samples_per_iteration = -1, \n                classification_stop = -1)\n                \n"

## Create Submission

In [26]:
# create time stamp
import re
import time
time_stamp = re.sub('[: ]', '_', time.asctime())

# score unlabeled test data
sub = mnist_model.predict(test)

# save file for submission
sub = sub['predict']

import numpy as np # create ID column
sub = h2o.H2OFrame(np.arange(1, 28001)).cbind(sub) 

sub.columns = ['ImageId', 'Label']

print(sub.head())

sub_fname = str(time_stamp) + '.csv'
h2o.download_csv(sub, sub_fname)

deeplearning prediction progress: |███████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


ImageId,Label
1,4
2,4
3,4
4,7
5,4
6,7
7,4
8,7
9,4
10,7


In [15]:
# shutdown h2o - this will erase all your unsaved frames and models in H2O
h2o.cluster().shutdown(prompt=True)

Are you sure you want to shutdown the H2O instance running at http://127.0.0.1:54321 (Y/N)? y
H2O session _sid_b711 closed.
